# Lesson 1: Simple ReAct Agent from Scratch

The code around the LLM is called the runtime

ReAct:<br/>
The LLM thinks what to do<br/>
LLM decides on action<br/>
Action is executed<br/>
Observation is fed to LLM<br/>
REPEAT<br/>

This notebook shows how to create an Agent using open ai API and python code. Next notebook: how to do same using langgraph.



In [ ]:
# based on https://til.simonwillison.net/llms/python-react-pattern

In [22]:
import openai
import re
import httpx
import os
from dotenv import load_dotenv

_ = load_dotenv()
from openai import OpenAI

In [23]:
client = OpenAI()

In [24]:
chat_completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Hello world"}]
)

In [26]:
#To ensure that it is working
chat_completion.choices[0].message.content

'Hello! How can I assist you today?'

In [ ]:
class Agent:
    """
    The Agent class uses Python's __call__ magic method (dunder method)
    to make instances callable like functions.
    
    What are dunder methods?
    - Methods with double underscores (__method__) are "magic methods" or "special methods"
    - They define how objects behave with built-in Python operations
    - Examples: __init__, __str__, __len__, __call__, etc.
    
    What does __call__ do?
    - Makes an instance callable (you can use it like a function)
    - Instead of: agent.send_message("Hello")
    - You can do: agent("Hello")
    """
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        """
        This method makes the Agent instance callable.
        When you do: agent("Hello"), Python calls this method.
        
        Without __call__, you'd need: agent.send_message("Hello")
        With __call__, you can do: agent("Hello") - cleaner syntax!
        """
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
                        model="gpt-4o", 
                        temperature=0,
                        messages=self.messages)
        return completion.choices[0].message.content
    

In [58]:
# Demonstration: Understanding __call__

# Example 1: Without __call__ (traditional way)
class TraditionalAgent:
    def send_message(self, message):
        return f"Received: {message}"

traditional = TraditionalAgent()
result1 = traditional.send_message("Hello")  # Must use method name
print("Traditional way:", result1)

# Example 2: With __call__ (callable instance)
class CallableAgent:
    def __call__(self, message):
        return f"Received: {message}"

callable_agent = CallableAgent()
result2 = callable_agent("Hello")  # Can call it like a function!
print("With __call__:", result2)

# Example 3: The Agent class in action
# When you create: abot = Agent(prompt)
# Then call: abot("How much does a toy poodle weigh?")
# Python internally does: abot.__call__("How much does a toy poodle weigh?")

print("\n" + "="*60)
print("Key Point: __call__ makes objects behave like functions!")
print("This is useful for classes that represent 'callable' concepts,")
print("like agents, processors, or transformers.")


Traditional way: Received: Hello
With __call__: Received: Hello

Key Point: __call__ makes objects behave like functions!
This is useful for classes that represent 'callable' concepts,
like agents, processors, or transformers.


# React agent system prompt

In [29]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

# Tools made available

In [30]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier": 
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [31]:
abot = Agent(prompt)

In [32]:
result = abot("How much does a toy poodle weigh?")
print(result)

Thought: I should look up the average weight of a Toy Poodle using the average_dog_weight action.
Action: average_dog_weight: Toy Poodle
PAUSE


Manually generating the obeservation

In [39]:
result = average_dog_weight("Toy Poodle")

In [40]:
result

'a toy poodles average weight is 7 lbs'

In [41]:
next_prompt = "Observation: {}".format(result)

In [42]:
abot(next_prompt)

'Answer: A Toy Poodle weighs an average of 7 lbs.'

In [43]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user', 'content': 'How much does a 

# Reinitialize agent to clear out messages


In [45]:
abot = Agent(prompt)

In [46]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
abot(question)

'Thought: I need to find the average weight of both a Border Collie and a Scottish Terrier, then add them together to find the combined weight.\nAction: average_dog_weight: Border Collie\nPAUSE'

In [47]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)

Observation: a Border Collies average weight is 37 lbs


In [48]:
abot(next_prompt)

'Action: average_dog_weight: Scottish Terrier\nPAUSE'

In [49]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)

Observation: Scottish Terriers average 20 lbs


In [50]:
abot(next_prompt)

'Action: calculate: 37 + 20\nPAUSE'

In [51]:
next_prompt = "Observation: {}".format(eval("37 + 20"))
print(next_prompt)

Observation: 57


In [52]:
abot(next_prompt)

'Answer: The combined average weight of a Border Collie and a Scottish Terrier is 57 lbs.'

### Add loop - such that we do not do this manually

Used to parse output to see if final or tool call

In [55]:
action_re = re.compile('^Action: (\w+): (.*)$')   # python regular expression to selection action

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
C:\Users\SPYROSDP\AppData\Local\Temp\ipykernel_3216\2389047760.py:1: SyntaxWarning: invalid escape sequence '\w'
  action_re = re.compile('^Action: (\w+): (.*)$')   # python regular expression to selection action


## Understanding the Regex Pattern

The regex pattern `action_re` is used to **parse the LLM's output** and extract action commands.

**Pattern breakdown:**
```python
action_re = re.compile(r'^Action: (\w+): (.*)$')
```

**What each part means:**
- `^` - Start of the line (must match from beginning)
- `Action: ` - Literal text "Action: " (exact match)
- `(\w+)` - **First capture group**: One or more word characters (letters, digits, underscore)
  - This captures the action name (e.g., "calculate", "average_dog_weight")
- `: ` - Literal ": " (colon and space)
- `(.*)` - **Second capture group**: Any characters (except newline), zero or more times
  - This captures the action input/parameters
- `$` - End of the line (must match to the end)

**What it matches:**
- ✅ `Action: calculate: 4 * 7 / 3`
- ✅ `Action: average_dog_weight: Bulldog`
- ❌ `Thought: I need to calculate this` (doesn't start with "Action:")
- ❌ `Action calculate 5` (missing colon after "Action")

**How it's used:**
When the LLM outputs something like:
```
Thought: I need to find the dog's weight
Action: average_dog_weight: Toy Poodle
PAUSE
```

The regex extracts:
- Group 1: `"average_dog_weight"` (the action name)
- Group 2: `"Toy Poodle"` (the action input)


In [60]:
# Demonstration: How the regex works

# Example LLM outputs
example_outputs = [
    "Thought: I need to calculate this\nAction: calculate: 4 * 7 / 3\nPAUSE",
    "Thought: Find dog weight\nAction: average_dog_weight: Bulldog\nPAUSE",
    "Thought: This is just a thought, no action",
    "Action calculate 5",  # Invalid - missing colon
]

print("Testing regex pattern on example outputs:\n")
for i, output in enumerate(example_outputs, 1):
    print(f"Example {i}:")
    print(f"  Output: {repr(output)}")
    lines = output.split('\n')
    for line in lines:
        match = action_re.match(line)
        if match:
            action_name, action_input = match.groups()
            print(f"  ✓ MATCHED: Action='{action_name}', Input='{action_input}'")
        else:
            if line.strip():  # Only print non-empty lines
                print(f"  ✗ No match: {line}")
    print()


Testing regex pattern on example outputs:

Example 1:
  Output: 'Thought: I need to calculate this\nAction: calculate: 4 * 7 / 3\nPAUSE'
  ✗ No match: Thought: I need to calculate this
  ✓ MATCHED: Action='calculate', Input='4 * 7 / 3'
  ✗ No match: PAUSE

Example 2:
  Output: 'Thought: Find dog weight\nAction: average_dog_weight: Bulldog\nPAUSE'
  ✗ No match: Thought: Find dog weight
  ✓ MATCHED: Action='average_dog_weight', Input='Bulldog'
  ✗ No match: PAUSE

Example 3:
  Output: 'Thought: This is just a thought, no action'
  ✗ No match: Thought: This is just a thought, no action

Example 4:
  Output: 'Action calculate 5'
  ✗ No match: Action calculate 5



## The `query()` Function - Automated ReAct Loop

The `query()` function **automates the ReAct (Reasoning + Acting) loop** that you were doing manually earlier.

**What it does:**
1. **Takes a question** and creates a fresh Agent
2. **Loops automatically** (up to `max_turns` times, default 5)
3. **Each iteration:**
   - Sends the prompt to the LLM
   - Parses the response to find action commands
   - If an action is found: executes it and feeds the observation back
   - If no action: returns (final answer reached)

**The ReAct Loop:**
```
Question → LLM → [Thought + Action] → Execute Action → Observation → LLM → [Answer]
                                                                    ↑________|
                                                                    (loop continues)
```

**Key Steps:**
- **Line 7**: `result = bot(next_prompt)` - Get LLM response
- **Lines 9-13**: Use regex to find action commands in the response
- **Lines 14-22**: If action found, execute it and create observation
- **Line 23**: If no action, return (we got the final answer!)

This replaces the manual process of:
1. Calling `abot(question)`
2. Manually extracting the action
3. Running the function
4. Creating `next_prompt = "Observation: ..."`
5. Calling `abot(next_prompt)` again
~

In [ ]:
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        # Step 2: Parse the response to find action commands
        # Split response into lines and check each line against the regex pattern
        # The regex looks for lines matching: "Action: action_name: input"
        actions = [
            action_re.match(a)  # Try to match regex pattern on this line
            for a in result.split('\n')  # Split response into individual lines
            if action_re.match(a)  # Only keep lines that match the pattern
        ]
        #if list is not empty
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            #wrong actions outputted by LLM
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

In [57]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
query(question)

Thought: I need to find the average weight of a Border Collie and a Scottish Terrier, then add them together to find their combined weight.
Action: average_dog_weight: Border Collie
PAUSE
 -- running average_dog_weight Border Collie
Observation: a Border Collies average weight is 37 lbs
Action: average_dog_weight: Scottish Terrier
PAUSE
 -- running average_dog_weight Scottish Terrier
Observation: Scottish Terriers average 20 lbs
Thought: Now that I have the average weights of both dogs, I can calculate their combined weight by adding them together.
Action: calculate: 37 + 20
PAUSE
 -- running calculate 37 + 20
Observation: 57
Answer: The combined average weight of a Border Collie and a Scottish Terrier is 57 lbs.
